# **Example for Paper**: [Non-Euclidean Universal Approximation](https://arxiv.org/abs/2006.02341)

## Preping

We compare three models in this implementation.  Each are feed-forward networks of the same dimensions:
- **Good model**: repsects our assumptions
- **Bad model**: does not
- **Vanilla model**: is a naive feed-forward benchmark
#### Import Libraries

In [50]:
# Alert(s)
import smtplib

# CV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelBinarizer

# DL: Tensorflow
import tensorflow as tf
from keras.utils.layer_utils import count_params
from tensorflow.python.framework import ops # Custome Tensorflow Functions
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Dense, Input
# DL: Tensorflow - Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

# Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Formatting:
import pandas as pd
import numpy as np

# Pre-Processing
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from scipy.special import expit

# Random Forest & Gradient Boosting (Arch. Construction)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

# Structuring
from pathlib import Path

# Visulatization
import matplotlib.pyplot as plt

# Writing, Reading, Exporting, and Importing
#from sklearn.externals import joblib
import pickle

# Timing
import time

# Misc
import gc
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import os

# Set-Seed
np.random.seed(2020)

#### Load Externally-Defined Functions

In [51]:
# Main Training Utility
exec(open('Helper_Utility.py').read())
# Helper Functions Utility
exec(open('Optimal_Deep_Feature_and_Readout_Util.py').read())
# Extra Utilities
exec(open('Grid_Enhanced_Network.py').read())

#### Load Data

In [52]:
# load dataset
data_path = "./data/housing_complete.csv"
X = pd.read_csv(data_path)

# Parse/Prepare Data
X_train, y_train, X_test, y_test= prepare_data(data_path, True)

#### Check and Make Paths

In [53]:
Path('./outputs/models/').mkdir(parents=True, exist_ok=True)
Path('./outputs/models/Benchmarks/Vanilla/').mkdir(parents=True, exist_ok=True)
Path('./outputs/models/Benchmarks/Bad/').mkdir(parents=True, exist_ok=True)
Path('./outputs/models/Deep_Features/Good_I/').mkdir(parents=True, exist_ok=True)
Path('./outputs/models/Deep_Features/Good_II/').mkdir(parents=True, exist_ok=True)
Path('./outputs/tables/').mkdir(parents=True, exist_ok=True)
Path('./outputs/results/').mkdir(parents=True, exist_ok=True)

---
---
---

# Good Model $I$:
Build and train the good model:
$$
\rho \circ f\circ \phi:\mathbb{R}^m\rightarrow \mathbb{R}^n.
$$
 - $f$ is a shallow feed-forward network with ReLU activation.  
 - Readout: $\rho(x) = \operatorname{Leaky-ReLU}\bullet (\exp(\tilde{A}_n)x+\tilde{b}_n)\circ \dots \circ \operatorname{Leaky-ReLU}\bullet (\exp(\tilde{A}_1)x+\tilde{b}_1)$
 - Feature Map: $\phi(x) = \operatorname{Leaky-ReLU}\bullet (\exp(A_n)x+b_n)\circ \dots \circ\operatorname{Leaky-ReLU}\bullet (\exp(A_1)x+b_1)$

where $A_i,\tilde{A}_j$ are square matrices.  


The matrices $\exp(A_i)$, and $\exp(\tilde{A}_i)$ are therefore invertible since $\exp$ maps any square matrix into the associated [General Linear Group](https://en.wikipedia.org/wiki/General_linear_group).  

In [54]:
#------------------------------------------------------------------------------------------------#
#                                      Define Predictive Model                                   #
#------------------------------------------------------------------------------------------------#

def def_trainable_layers_Nice_Input_Output(height, Depth_Feature_Map, Depth_Readout_Map, learning_rate, input_dim, output_dim):
    #----------------------------#
    # Maximally Interacting Layer #
    #-----------------------------#
    # Initialize Inputs
    input_layer = tf.keras.Input(shape=(input_dim,))
    
    
    #------------------#
    # Deep Feature Map #
    #------------------#
    for i_feature_depth in range(Depth_Feature_Map):
        # First Layer
        if i_feature_depth == 0:
            deep_feature_map = fullyConnected_Dense_Invertible(input_dim)(input_layer)
            deep_feature_map = tf.nn.leaky_relu(deep_feature_map)
        else:
            deep_feature_map = fullyConnected_Dense_Invertible(input_dim)(deep_feature_map)
            deep_feature_map = tf.nn.leaky_relu(deep_feature_map)
    
    #------------------#
    #   Core Layers    #
    #------------------#
    core_layers = fullyConnected_Dense(height)(deep_feature_map)
    # Activation
    core_layers = tf.nn.relu(core_layers)
    # Affine Layer (Dense Fully Connected)
    output_layers = fullyConnected_Dense(output_dim)(core_layers)
    
    
    #------------------#
    #  Readout Layers  #
    #------------------#   
    for i_depth_readout in range(Depth_Readout_Map):
        # First Layer
        if i_depth_readout == 0:
            output_layers = fullyConnected_Dense_Invertible(output_dim)(output_layers)
            output_layers = tf.nn.leaky_relu(output_layers)
        else:
            output_layers = fullyConnected_Dense_Invertible(output_dim)(output_layers)
            output_layers = tf.nn.leaky_relu(output_layers)
    
    
    # Define Input/Output Relationship (Arch.)
    trainable_layers_model = tf.keras.Model(input_layer, output_layers)
    
    
    #----------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)
    trainable_layers_model.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])

    return trainable_layers_model

#------------------------------------------------------------------------------------------------#
#                                      Build Predictive Model                                    #
#------------------------------------------------------------------------------------------------#

def build_and_predict_nice_model(n_folds , n_jobs):

    # Deep Feature Network
    Nice_Model_CV = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=def_trainable_layers_Nice_Input_Output, verbose=True)
    
    # Randomized CV
    Nice_Model_CVer = RandomizedSearchCV(estimator=Nice_Model_CV, 
                                    n_jobs=n_jobs,
                                    cv=KFold(CV_folds, random_state=2020, shuffle=True),
                                    param_distributions=param_grid_Nice_Nets,
                                    n_iter=n_iter,
                                    return_train_score=True,
                                    random_state=2020,
                                    verbose=10)
    
    # Fit
    Nice_Model_CVer.fit(X_train,y_train)

    # Write Predictions
    y_hat_train = Nice_Model_CVer.predict(X_train)
    y_hat_test = Nice_Model_CVer.predict(X_test)
    
    # Return Values
    return y_hat_train, y_hat_test

# Update User
#-------------#
print('Built Mode: <Good I>')

Built Mode: <Good I>


### Make Predictions

In [55]:
# Initialize & User Updates
#--------------------------#
y_hat_train_good, y_hat_test_good = build_and_predict_nice_model(n_folds = 2, n_jobs = 2)
print('Cross-Validated Model: <Good I>')

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   16.6s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   16.6s finished


Epoch 1/2
452/452 [==============================] - 3s 6ms/step - loss: 1.4207 - mse: 3.4930 - mae: 1.4207 - mape: 69.3144
Epoch 2/2
194/194 [==============================] - 0s 2ms/step
Cross-Validated Model: <Good I>


# Good Model $II$:
Build and train the good model:
$$
\rho \circ f\circ \phi:\mathbb{R}^m\rightarrow \mathbb{R}^n.
$$
 - $f$ is a shallow feed-forward network with ReLU activation.  
 - Readout: $\rho(x) = \operatorname{Leaky-ReLU}\bullet (\exp(\tilde{A}_n)x+\tilde{b}_n)\circ \dots \circ \operatorname{Leaky-ReLU}\bullet (\exp(\tilde{A}_1)x+\tilde{b}_1)$
 - Feature Map: $\phi(x) = (x,\operatorname{Leaky-ReLU}\bullet (\exp(A_n)x+b_n)\circ \dots \circ\operatorname{Leaky-ReLU}\bullet (\exp(A_1)x+b_1))$,

where $A_i,\tilde{A}_j$ are square matrices, and $A_i,b_i$ are generated randomly by drawing their components from the standardized Bernoulli distribution.


The matrices $\exp(A_i)$, and $\exp(\tilde{A}_i)$ are therefore invertible since $\exp$ maps any square matrix into the associated [General Linear Group](https://en.wikipedia.org/wiki/General_linear_group).  

## Generate Random Deep Feature(s)

In [56]:
### Initialize Parameters
#------------------------#
# Initialize History
Randomized_Depth = np.random.poisson(2)
past_val = -1
current_position = 0
# Initalize Features
X_train_features = X_train
X_test_features = X_test

# Construct Deep Randomized Features
#------------------------------------#
# Set Seed
np.random.seed(2020)


# Builds Features
for i in range(N_Features):    
    # Transformations
    #-----------------#
    # Build
    if Randomized_Depth > 0:
        # Note: Write Non-Liearly Transformed Features only if transformation has been applied, only if Depth >0
        
        # Apply Activation
        X_train_features_loop = compositer(X_train_features)
        X_test_features_loop = compositer(X_test_features)
        # Apply Random Weights
        Weights_random = (np.random.binomial(1,.5,(X_train_features_loop.shape[1],X_train_features_loop.shape[1])) - .5)*2 # Generate Random Weights
        X_train_features_loop = np.matmul(X_train_features_loop,Weights_random)
        X_test_features_loop = np.matmul(X_test_features_loop,Weights_random)
        # # Apply Bias
        biases_random = (np.random.binomial(1,.5,X_train_features_loop.shape[1]) -.5)*2         # Generate Random Weights and Biases from Recentered Binomial Law
        X_train_features_loop = X_train_features_loop + biases_random
        X_test_features_loop = X_test_features_loop + biases_random
        
    else:
        X_train_features_loop = X_train_features
        X_test_features_loop = X_test_features

    # Update User #
    #-------------#
    print("Current Step: " +str((i+1)/N_Features))
    
# Coerce into nice form:
X_train_features = pd.DataFrame(X_train_features)
X_test_features = pd.DataFrame(X_test_features)
X_train.reset_index(drop=True, inplace=True)
X_train_features.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
X_test_features.reset_index(drop=True, inplace=True)

# Create Features
Random_Feature_Space_train = pd.concat([X_train,X_train_features],axis=1)
Random_Feature_Space_test = pd.concat([X_test,X_test_features],axis=1)

# Update User #
#-------------#
print('Generated Features: Done!')
print(Random_Feature_Space_train.head())
print(Random_Feature_Space_test.head())

Current Step: 0.25
Current Step: 0.5
Current Step: 0.75
Current Step: 1.0
Generated Features: Done!
   longitude  latitude  housing_median_age  total_rooms  population  \
0    -122.28     37.90                52.0       2261.0       819.0   
1    -122.28     37.75                20.0       1156.0       583.0   
2    -122.35     37.93                41.0        268.0       198.0   
3    -122.44     37.76                30.0       5089.0      1935.0   
4    -122.46     37.76                28.0       1072.0       363.0   

   households  median_income        x1        x2        x3  longitude  \
0       335.0         4.9083  0.004823  0.020925  0.097437    -122.28   
1       326.0         3.1972 -0.010159  0.017270 -0.051777    -122.28   
2        82.0         3.2222 -0.009778  0.087640  0.127816    -122.35   
3      1139.0         4.6053 -0.050311  0.171876 -0.040424    -122.44   
4       168.0         6.1636 -0.055439  0.191173 -0.040088    -122.46   

   latitude  housing_median_age  t

## Train DNN Model

In [57]:
# Reload Grid
exec(open('Grid_Enhanced_Network.py').read())
# Adjust Input Space's Dimension
param_grid_Nice_Nets['input_dim'] = [Random_Feature_Space_train.shape[1]]

def def_trainable_layers_Randomized_Feature(height, Depth_Feature_Map, Depth_Readout_Map, learning_rate, input_dim, output_dim):
    #----------------------------#
    # Maximally Interacting Layer #
    #-----------------------------#
    # Initialize Inputs
    input_layer = tf.keras.Input(shape=(input_dim,))
    
    
    #------------------#
    #   Core Layers    #
    #------------------#
    core_layers = fullyConnected_Dense(height)(input_layer)
    # Activation
    core_layers = tf.nn.relu(core_layers)
    # Affine Layer (Dense Fully Connected)
    output_layers = fullyConnected_Dense(output_dim)(core_layers)
    
    
    #------------------#
    #  Readout Layers  #
    #------------------#   
#     for i_depth_readout in range(Depth_Readout_Map):
#         # First Layer
#         if i_depth_readout == 0:
#             output_layers = fullyConnected_Dense_Invertible(output_dim)(output_layers)
#             output_layers = tf.nn.leaky_relu(output_layers)
#         else:
#             output_layers = fullyConnected_Dense_Invertible(output_dim)(output_layers)
#             output_layers = tf.nn.leaky_relu(output_layers)
    
    
    # Define Input/Output Relationship (Arch.)
    trainable_layers_model = tf.keras.Model(input_layer, output_layers)
    
    
    #----------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)
    trainable_layers_model.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])

    return trainable_layers_model

#------------------------------------------------------------------------------------------------#
#                                      Build Predictive Model                                    #
#------------------------------------------------------------------------------------------------#

def build_and_predict_nice_modelII(n_folds , n_jobs):

    # Deep Feature Network
    Nice_Model_CV = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=def_trainable_layers_Randomized_Feature, verbose=True)
    
    # Randomized CV
    Nice_Model_CVer = RandomizedSearchCV(estimator=Nice_Model_CV, 
                                    n_jobs=n_jobs,
                                    cv=KFold(CV_folds, random_state=2020, shuffle=True),
                                    param_distributions=param_grid_Nice_Nets,
                                    n_iter=n_iter,
                                    return_train_score=True,
                                    random_state=2020,
                                    verbose=10)
    
    # Fit
    Nice_Model_CVer.fit(Random_Feature_Space_train,y_train)

    # Write Predictions
    y_hat_train = Nice_Model_CVer.predict(Random_Feature_Space_train)
    y_hat_test = Nice_Model_CVer.predict(Random_Feature_Space_test)
    
    # Return Values
    return y_hat_train, y_hat_test

# Update User
#-------------#
print('Built Mode: <Good II>')

Built Mode: <Good II>


### Make Predictions

In [58]:
# Initialize & User Updates
#--------------------------#
y_hat_train_goodII, y_hat_test_goodII = build_and_predict_nice_modelII(n_folds = 2, n_jobs = 2)
print('Cross-Validated Model: "Good II"')

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    6.5s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    6.5s finished


Epoch 1/2
452/452 [==============================] - 1s 1ms/step - loss: 2.0939 - mse: 5.7222 - mae: 2.0939 - mape: 101.3458
Epoch 2/2
194/194 [==============================] - 0s 726us/step
Cross-Validated Model: "Good II"


---
---
---

---
# Benchmark(s)
---

#### Reload CV Grid

In [43]:
exec(open('Grid_Enhanced_Network.py').read())

---
---
---

# Bad Model:
Build and train the *bad* model:
$$
\rho \circ f\circ \phi:\mathbb{R}^m\rightarrow \mathbb{R}^n.
$$
 - $f$ is a shallow feed-forward network with ReLU activation.  
 - Readout: $\rho(x) = \operatorname{ReLU}\bullet (\exp(\tilde{A}_n)x+\tilde{b}_n)\circ \dots \circ \operatorname{ReLU}\bullet (\exp(\tilde{A}_1)x+\tilde{b}_1)$
 - Feature Map: $\phi(x) = \operatorname{ReLU}\bullet (\exp(A_n)x+b_n)\circ \dots \circ\operatorname{ReLU}\bullet (\exp(A_1)x+b_1)$

where $A_i,\tilde{A}_j$ are square matrices.  The maps $\rho$ and $\phi$ are neither injective nor are they surjective since $x\mapsto \operatorname{ReLU}(x)$ is neither injective nor surjective as a map from $\mathbb{R}^k$ to $\mathbb{R}^k$; where $m=n$.  

*Note*:  The key point here is that the input and output maps are forced to be of the same dimension.  Note that, this also violates the minimal bounds derivated in [this paper](https://arxiv.org/abs/1710.11278) for deep ReLU networks.  

In [44]:
#------------------------------------------------------------------------------------------------#
#                                      Define Predictive Model                                   #
#------------------------------------------------------------------------------------------------#

def def_trainable_layers_Bad_Input_Output(height, Depth_Feature_Map, Depth_Readout_Map, learning_rate, input_dim,output_dim):
    #----------------------------#
    # Maximally Interacting Layer #
    #-----------------------------#
    # Initialize Inputs
    input_layer = tf.keras.Input(shape=(input_dim,))
    
    
    #------------------#
    # Deep Feature Map #
    #------------------#
    for i_feature_depth in range(Depth_Feature_Map):
        # First Layer
        if i_feature_depth == 0:
            deep_feature_map = fullyConnected_Dense(input_dim)(input_layer)
            deep_feature_map = tf.nn.relu(deep_feature_map)
        else:
            deep_feature_map = fullyConnected_Dense(input_dim)(deep_feature_map)
            deep_feature_map = tf.nn.relu(deep_feature_map)
    
    #------------------#
    #   Core Layers    #
    #------------------#
    core_layers = fullyConnected_Dense(height)(deep_feature_map)
    # Activation
    core_layers = tf.nn.relu(core_layers)
    # Affine Layer (Dense Fully Connected)
    output_layers = fullyConnected_Dense(output_dim)(core_layers)
    
    
    #------------------#
    #  Readout Layers  #
    #------------------#   
    for i_depth_readout in range(Depth_Readout_Map):
        # First Layer
        if i_depth_readout == 0:
            output_layers = fullyConnected_Dense(output_dim)(output_layers)
            output_layers = tf.nn.relu(output_layers)
        else:
            output_layers = fullyConnected_Dense(output_dim)(output_layers)
            output_layers = tf.nn.relu(output_layers)
    
    
    # Define Input/Output Relationship (Arch.)
    trainable_layers_model = tf.keras.Model(input_layer, output_layers)
    
    
    #----------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)
    trainable_layers_model.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])

    return trainable_layers_model

#------------------------------------------------------------------------------------------------#
#                                      Build Predictive Model                                    #
#------------------------------------------------------------------------------------------------#

def build_and_predict_bad_model(n_folds , n_jobs):

    # Deep Feature Network
    Bad_Model_CV = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=def_trainable_layers_Bad_Input_Output, verbose=True)
    
    # Randomized CV
    Bad_Model_CVer = RandomizedSearchCV(estimator=Bad_Model_CV, 
                                    n_jobs=n_jobs,
                                    cv=KFold(CV_folds, random_state=2020, shuffle=True),
                                    param_distributions=param_grid_Nice_Nets,
                                    n_iter=n_iter,
                                    return_train_score=True,
                                    random_state=2020,
                                    verbose=10)
    
    # Fit
    Bad_Model_CVer.fit(X_train,y_train)

    # Write Predictions
    y_hat_train = Bad_Model_CVer.predict(X_train)
    y_hat_test = Bad_Model_CVer.predict(X_test)
    
    # Return Values
    return y_hat_train, y_hat_test

# Update User
#-------------#
print('Built Bad Model')

Built Bad Model


In [45]:
# Initialize & User Updates
#--------------------------#
y_hat_train_bad, y_hat_test_bad = build_and_predict_bad_model(n_folds = 2, n_jobs = 2)
print('Cross-Validated: Vanilla Model')

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.2s finished


Epoch 1/2
452/452 [==============================] - 1s 1ms/step - loss: 1.9985 - mse: 5.3318 - mae: 1.9985 - mape: 95.0137
Epoch 2/2
194/194 [==============================] - 0s 662us/step
Cross-Validated: Vanilla Model


---

# Benchmark ffNN Model (Vanilla)

---

In [46]:
#------------------------------------------------------------------------------------------------#
#                                      Define Predictive Model                                   #
#------------------------------------------------------------------------------------------------#

def def_trainable_layers_Vanilla(height, Depth_Feature_Map, Depth_Readout_Map, learning_rate, input_dim, output_dim):
    #----------------------------#
    # Maximally Interacting Layer #
    #-----------------------------#
    # Initialize Inputs
    input_layer = tf.keras.Input(shape=(input_dim,))
    
    #------------------#
    #   Core Layers    #
    #------------------#
    core_layers = fullyConnected_Dense(height)(input_layer)
    # Activation
    core_layers = tf.nn.relu(core_layers)
    # Affine Layer (Dense Fully Connected)
    output_layers = fullyConnected_Dense(output_dim)(core_layers)
    
    
    # Define Input/Output Relationship (Arch.)
    trainable_layers_model = tf.keras.Model(input_layer, output_layers)
    
    
    #----------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)
    trainable_layers_model.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])

    return trainable_layers_model

#------------------------------------------------------------------------------------------------#
#                                      Build Predictive Model                                    #
#------------------------------------------------------------------------------------------------#

def build_and_predict_Vanilla_model(n_folds , n_jobs):

    # Deep Feature Network
    Vanilla_Model_CV = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=def_trainable_layers_Vanilla, verbose=True)
    
    # Randomized CV
    Vanilla_Model_CVer = RandomizedSearchCV(estimator=Vanilla_Model_CV, 
                                    n_jobs=n_jobs,
                                    cv=KFold(CV_folds, random_state=2020, shuffle=True),
                                    param_distributions=param_grid_Nice_Nets,
                                    n_iter=n_iter,
                                    return_train_score=True,
                                    random_state=2020,
                                    verbose=10)
    
    # Fit
    Vanilla_Model_CVer.fit(X_train,y_train)

    # Write Predictions
    y_hat_train = Vanilla_Model_CVer.predict(X_train)
    y_hat_test = Vanilla_Model_CVer.predict(X_test)
    
    # Return Values
    return y_hat_train, y_hat_test

# Update User
#-------------#
print('Built Vanilla Model')

Built Vanilla Model


### Make Predictions

In [47]:
# Initialize & User Updates
#--------------------------#
y_hat_train_Vanilla, y_hat_test_Vanilla = build_and_predict_Vanilla_model(n_folds = 2, n_jobs = 2)
print('Cross-Validated: Vanilla Model')

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.1s finished


Epoch 1/2
452/452 [==============================] - 0s 946us/step - loss: 2.0052 - mse: 5.3588 - mae: 2.0052 - mape: 95.4514
Epoch 2/2
194/194 [==============================] - 0s 603us/step
Cross-Validated: Vanilla Model


# Record Predictions/ Comparisons
Generate Classes

In [63]:
# Benchmark Models #
#------------------#
# Results with Good I Model
Perform_GoodI = reporter(y_hat_train_good,y_hat_test_good,y_train,y_test)
# Results with Good II Model
Perform_GoodII = reporter(y_hat_train_goodI,y_hat_test_goodII,y_train,y_test)


# Benchmark Models Performance #
#------------------------------#
# Results with Bad Model
Perform_Bad = reporter(y_hat_train_bad,y_hat_test_bad,y_train,y_test)
# Results Vanilla
Perform_Vanilla = reporter(y_hat_train_Vanilla,y_hat_test_Vanilla,y_train,y_test)

In [78]:
# Performance Metrics
#----------------------#
performance_train = pd.DataFrame({
                    'Good I': Perform_GoodI.train,
                    'Good II': Perform_GoodII.train,
                    'Bad': Perform_Bad.train,
                    'Vanilla': Perform_Vanilla.train})

performance_test = pd.DataFrame({
                    'Good I': Perform_GoodI.test,
                    'Good II': Perform_GoodII.test,
                    'Bad': Perform_Bad.test,
                    'Vanilla': Perform_Vanilla.test})

# Write Results
#---------------#
# LaTeX
performance_train.to_latex('./outputs/results/Performance_train.txt')
performance_test.to_latex('./outputs/results/Performance_test.txt')
# Write to Txt
cur_path = os.path.expanduser('./outputs/results/Performance_train_text.txt')
with open(cur_path, "w") as f:
    f.write(str(performance_train))
cur_path = os.path.expanduser('./outputs/results/Performance_test_text.txt')
with open(cur_path, "w") as f:
    f.write(str(performance_test))

# Live Readings

In [85]:
print('Et-Voila!')
print(' ')
print(' ')
print('#-------------------#')
print(' PERFORMANCE SUMMARY:')
print('#-------------------#')
print(' ')
print(' ')
print('---------------------')
print('Training Performance')
print('---------------------')
print('-------------------------------------------------------------')
print(performance_train)
print('-------------------------------------------------------------')
print('---------------------')
print('Testing Performance')
print('---------------------')
print('-------------------------------------------------------------')
print(performance_test)
print('-------------------------------------------------------------')
print(' ')
print(' ')
print('Fin')

Et-Voila!
 
 
#-------------------#
 PERFORMANCE SUMMARY:
#-------------------#
 
 
---------------------
Training Performance
---------------------
-------------------------------------------------------------
         Good I      Good II          Bad      Vanilla
MAE    1.021502     1.931493     1.909398     1.937359
MSE    2.002546     5.068936     4.982967     5.090549
MAPE  77.900601  1353.623117  1162.452322  1421.507214
-------------------------------------------------------------
---------------------
Testing Performance
---------------------
-------------------------------------------------------------
         Good I      Good II          Bad      Vanilla
MAE    1.012621     2.009092     1.892434     1.920396
MSE    1.992302     5.354643     4.899480     5.006113
MAPE  76.746722  4221.398045  1152.124826  1409.061450
-------------------------------------------------------------
 
 
Fin


---
#### 😊 Fin 😊
---